# Построение воходных конвейеров с использованием tf.data - API-интерфейса Dataset библиотеки

Когда набор данны слишком большой и потому не умущается в памяти, нам понадобится загружать данные из основного устройства хранения порциями, т.е. пакет за пакетом.

## Создание объекта Dataset из существующих тензоров

In [43]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
import numpy as np
np.set_printoptions(precision=3)

In [44]:
a = [1.2, 3.4, 7.5, 4.1, 5.0, 1.0]
ds = tf.data.Dataset.from_tensor_slices(a)
print(ds)

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.float32, name=None)>


Функция возвратила объект класса `Dataset`, который можно применить для прохода по индивидуальным элементам во входном наборе данных.

In [45]:
for item in ds:
    print(item)

tf.Tensor(1.2, shape=(), dtype=float32)
tf.Tensor(3.4, shape=(), dtype=float32)
tf.Tensor(7.5, shape=(), dtype=float32)
tf.Tensor(4.1, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)


Создатим из этого набора данных пакеты с размером 2:

In [46]:
ds_batch = ds.batch(2)
for i, elem in enumerate(ds_batch, 1):
    print('пакет {}:' .format(i), elem.numpy())

пакет 1: [1.2 3.4]
пакет 2: [7.5 4.1]
пакет 3: [5. 1.]


Если количество элементов в тензоре не желаемому зармеру пакета (batch):

In [47]:
ds_batch = ds.batch(4, drop_remainder=False)
for i, elem in enumerate(ds_batch, 1):
    print('пакет {}:' .format(i), elem.numpy())

пакет 1: [1.2 3.4 7.5 4.1]
пакет 2: [5. 1.]


In [48]:
ds_batch = ds.batch(4, drop_remainder=True)
for i, elem in enumerate(ds_batch, 1):
    print('пакет {}:' .format(i), elem.numpy())

пакет 1: [1.2 3.4 7.5 4.1]


## Объединение двух тензоров в общий набор данных

Часто данные находятся в двух или большем числе тензоров, напимер, тензор признаков и тензор меток. В таких случаях надо построить набор данных, который объединит эти тензоры и позволит извлекать элементы тензоров в кортежах.

In [49]:
tf.random.set_seed(1)
t_x = tf.random.uniform([4, 3], dtype=tf.float32)
t_y = tf.range(4)

print(t_x.numpy())
print(t_y.numpy())

[[0.165 0.901 0.631]
 [0.435 0.292 0.643]
 [0.976 0.435 0.66 ]
 [0.605 0.637 0.614]]
[0 1 2 3]


Создадим общий набор данных:
1. Создадим два набора данных
1. Построим общий набор данных, используя фукнцию `.zip()`

In [50]:
ds_x = tf.data.Dataset.from_tensor_slices(t_x)
ds_y = tf.data.Dataset.from_tensor_slices(t_y)
# Объединим объекты Dataset в один:
ds_joint = tf.data.Dataset.zip((ds_x, ds_y))

Тензоры объединеты в кортежи:

In [51]:
for example in ds_joint:
    print(example)

(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.165, 0.901, 0.631], dtype=float32)>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.435, 0.292, 0.643], dtype=float32)>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)
(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.976, 0.435, 0.66 ], dtype=float32)>, <tf.Tensor: shape=(), dtype=int32, numpy=2>)
(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.605, 0.637, 0.614], dtype=float32)>, <tf.Tensor: shape=(), dtype=int32, numpy=3>)


Поличим данные из кортежей:

In [52]:
for example in ds_joint:
    print('x:', example[0].numpy(),
          'y:', example[1].numpy())

x: [0.165 0.901 0.631] y: 0
x: [0.435 0.292 0.643] y: 1
x: [0.976 0.435 0.66 ] y: 2
x: [0.605 0.637 0.614] y: 3


Все это можно сделать, используя метод `tf.data.Dataset.from_tensor_slices():

In [53]:
ds_jount = tf.data.Dataset.from_tensor_slices((t_x, t_y))
for example in ds_joint:
    print('x:', example[0].numpy(),
          'y:', example[1].numpy())

x: [0.165 0.901 0.631] y: 0
x: [0.435 0.292 0.643] y: 1
x: [0.976 0.435 0.66 ] y: 2
x: [0.605 0.637 0.614] y: 3


## Применение трансформации к отдельным элементам набора данных

In [54]:
# Трансформируем набор признаков:
ds_trans = ds_joint.map(lambda x, y: (x*2-1, y))
for example in ds_trans:
    print(' x:', example[0].numpy(),
          ' y:', example[1].numpy())

 x: [-0.67   0.803  0.262]  y: 0
 x: [-0.131 -0.416  0.285]  y: 1
 x: [ 0.952 -0.13   0.32 ]  y: 2
 x: [0.21  0.273 0.229]  y: 3


## Тасование, создание пакетов и повторение

### Создание перетасованной версии набора данных

In [55]:
tf.random.set_seed(1)
ds = ds_joint.shuffle(buffer_size=len(t_x))
for example in ds:
    print(' x:', example[0].numpy(),
          ' y:', example[1].numpy())

 x: [0.976 0.435 0.66 ]  y: 2
 x: [0.435 0.292 0.643]  y: 1
 x: [0.165 0.901 0.631]  y: 0
 x: [0.605 0.637 0.614]  y: 3


Строки тасуются без потери соответствия 1:1 между элементами `x` и `y`

Аргумент `buffer_size` определяет количество элементов в наборе данных, группируемых перед тасованием. Если выбрать небольшое значение для `buffer_aize`, то нельзя будет обеспечить идеальное тасование набора данных

### Создание пакетов из объединенных данных

In [63]:
ds = ds_joint.batch(batch_size=2,
                    drop_remainder=False)

batch_x, batch_y = next(iter(ds))                    
print('Пакет-x:\n', batch_x.numpy(), batch_x.shape)

Пакет-x:
 [[0.165 0.901 0.631]
 [0.435 0.292 0.643]] (2, 3)


In [64]:
print('Пакет-y:\n', batch_y.numpy(), batch_y.shape)

Пакет-y:
 [0 1] (2,)


Повторим разбитый пакет набора данных три раза:

In [69]:
ds = ds_joint.batch(3).repeat(count=2)
for i, (batch_x, batch_y) in enumerate(ds):
    print(i, batch_x.shape, batch_y.numpy())

0 (3, 3) [0 1 2]
1 (1, 3) [3]
2 (3, 3) [0 1 2]
3 (1, 3) [3]


Если изменить порядок следования операций, то результат будет другим:

In [76]:
ds = ds_joint.repeat(count=2).batch(3)
for i, (batch_x, batch_y) in enumerate(ds):
    print(i, batch_x.shape, batch_y.numpy())

0 (3, 3) [0 1 2]
1 (3, 3) [3 0 1]
2 (2, 3) [2 3]


## Порядок 1

тасование --> создание пакетов --> повторение

In [83]:
tf.random.set_seed(1)
ds = ds_joint.shuffle(4).batch(2).repeat(3)
for i, (batch_x, batch_y) in enumerate(ds):
    print(i, batch_x.shape, batch_y.numpy())

0 (2, 3) [2 1]
1 (2, 3) [0 3]
2 (2, 3) [0 3]
3 (2, 3) [1 2]
4 (2, 3) [3 0]
5 (2, 3) [1 2]


## Порядок 2

создание пакетов --> тасование --> повторение

In [84]:
tf.random.set_seed(1)
ds = ds_joint.batch(2).shuffle(4).repeat(3)
for i, (batch_x, batch_y) in enumerate(ds):
    print(i, batch_x.shape, batch_y.numpy())

0 (2, 3) [0 1]
1 (2, 3) [2 3]
2 (2, 3) [0 1]
3 (2, 3) [2 3]
4 (2, 3) [2 3]
5 (2, 3) [0 1]


## Порядок 3

создание пакетов --> повторение --> тасование

In [79]:
tf.random.set_seed(1)
ds = ds_joint.batch(2).repeat(3).shuffle(4)
for i, (batch_x, batch_y) in enumerate(ds):
    print(i, batch_x.shape, batch_y.numpy())

0 (2, 3) [0 1]
1 (2, 3) [0 1]
2 (2, 3) [2 3]
3 (2, 3) [2 3]
4 (2, 3) [0 1]
5 (2, 3) [2 3]
